# Hyperparameter Tuning with the SageMaker TensorFlow Container

This tutorial focuses on how to create a convolutional neural network model to train the MNIST dataset using the SageMaker TensorFlow container. It leverages hyperparameter tuning to run multiple training jobs with different hyperparameter combinations, to find the one with the best model training result.


## 개요

- [작업 실행 시 필요한 라이브러리 import](#작업-실행-시-필요한-라이브러리-import)
- [SageMaker 세션과 Role, 사용 버킷 정의](#sagemaker-세션과-role-사용-버킷-정의)
- [학습에 사용할 데이터 준비](#학습에-사용할-데이터-준비)
- [Run a TensorFlow Training Job](#run-a-tensorflow-training-job)
- [Run a Hyperparameter Tuning Job](#run-a-hyperparameter-tuning-job)

## Reference

- https://sagemaker-examples.readthedocs.io/en/latest/hyperparameter_tuning/tensorflow2_mnist/hpo_tensorflow2_mnist_outputs.html


### 작업 실행 시 필요한 라이브러리 import


In [ ]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --upgrade boto3
!{sys.executable} -m pip install --upgrade sagemaker
!{sys.executable} -m pip install --upgrade tensorflow

In [ ]:
import boto3
import sagemaker

### SageMaker 세션과 Role, 사용 버킷 정의


In [ ]:
sagemaker_session = sagemaker.session.Session()
boto_session = boto3.Session()

role = sagemaker.get_execution_role()
# SageMaker사용시 log, model artifact 등의 저장을 위해 default로 지정되는 s3를 사용할 수도 있고,
# 아니면 직접 s3 세팅해서 이걸 사용하라고 지정해 줄 수도 있습니다.
# 실습에서는 default로 제공되는 s3를 사용해보겠습니다.

# default bucket이 아니라 직접 만든 s3 bucket을 사용할 수도 있습니다.
# 만약 직접 생성한 s3 bucket을 사용할 경우 에러가 난다면 아래의 두 경우일 확률이 높습니다.
# 1. bucket이 없는 경우. 즉, bucket을 생성하지 않았는데 그 bucket을 사용하려고 하는 경우
# 2. bucket을 생성했지만, role에 해당 bucket에 대한 read, write 권한이 없는 경우
default_bucket = sagemaker_session.default_bucket()

# sagemaker를 사용
sm = boto_session.client("sagemaker")
region = boto_session.region_name

### 학습에 사용할 데이터 준비

Download the MNIST data from a public S3 bucket and save it in a temporary directory.


In [ ]:
import os
import boto3

public_bucket = f"sagemaker-example-files-prod-{region}"
local_data_dir = "/tmp/data"


# Download training and testing data from a public S3 bucket
def download_from_s3(data_dir="/tmp/data", train=True):
    """Download MNIST dataset and convert it to numpy array

    Args:
        data_dir (str): directory to save the data
        train (bool): download training set

    Returns:
        None
    """
    # project root
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    if train:
        images_file = "train-images-idx3-ubyte.gz"
        labels_file = "train-labels-idx1-ubyte.gz"
    else:
        images_file = "t10k-images-idx3-ubyte.gz"
        labels_file = "t10k-labels-idx1-ubyte.gz"

    # download objects
    s3 = boto3.client("s3")
    bucket = public_bucket
    for obj in [images_file, labels_file]:
        key = os.path.join("datasets/image/MNIST", obj)
        dest = os.path.join(data_dir, obj)
        if not os.path.exists(dest):
            s3.download_file(bucket, key, dest)
    return


download_from_s3(local_data_dir, True)
download_from_s3(local_data_dir, False)

### Run a TensorFlow Training Job


In [ ]:
from sagemaker.tensorflow.estimator import TensorFlow
from sagemaker.experiments.run import Run

In [ ]:
# TensorFlow Estimator 생성
est = TensorFlow(
    entry_point="train.py",  # train.py: 학습에 사용할 코드. 이 코드는 code 폴더에 저장되어 있음
    source_dir="code",
    role=role,
    framework_version="2.3.1",
    # model_dir: remote에 있는 instance, 즉 sagemaker instance에 저장되는 model artifact가 저장되는 디렉토리
    model_dir="/opt/ml/model",
    py_version="py37",
    instance_type="ml.m5.xlarge",
    instance_count=1,
    volume_size=250,  # volue size 지정: 250GB
    hyperparameters={
        "batch-size": 512,
        "epochs": 4,
    },
)

In [ ]:
prefix = "mnist"
bucket = sagemaker_session.default_bucket()

"""
Uploads data from the local directory to an S3 bucket.

Args:
  local_data_dir (str): The local directory path where the data is located.

Returns:
  dict: A dictionary containing the S3 bucket location for the training and testing data.

"""
loc = sagemaker_session.upload_data(
    path=local_data_dir, bucket=bucket, key_prefix=prefix)

channels = {"training": loc, "testing": loc}

### Run a Hyperparameter Tuning Job


In [ ]:
# Hyperparameter Tuning
# Hyperparameter tuning을 위한 parameter range 설정
from sagemaker.tuner import ContinuousParameter, HyperparameterTuner

# learning rate를 1e-4 ~ 1e-3 사이의 값으로 설정
hyperparameter_range = {"learning-rate": ContinuousParameter(1e-4, 1e-3)}

In [ ]:
objective_metric_name = "average test loss"
# metic을 어떤 정책을 기반으로 최적화 할 것인지 설정
objective_type = "Minimize"  # 만약 average test loss를 높이고 싶으면 Maximize로 설정

metric_definitions = [
    {
        "Name": "average test loss",
        "Regex": "Test Loss: ([0-9\\.]+)",
    }
]

In [ ]:
tuner = HyperparameterTuner(
    est,
    objective_metric_name,
    hyperparameter_range,
    metric_definitions,
    max_jobs=3,
    max_parallel_jobs=3,
    objective_type=objective_type
)

tuner.fit(inputs=channels)